In [28]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import _thread
import os
from nltk import word_tokenize
import nltk
from string import punctuation
from collections import Counter

In [21]:
stopwords = nltk.corpus.stopwords.words('portuguese')
numbers = '0123456789'

In [43]:
def pre_processar(texto):
    texto = ' '.join([t for t in word_tokenize(texto.lower()) if (t not in stopwords) and (t not in punctuation + numbers)])
    return Counter(word_tokenize(texto))

In [126]:
def contar_busca(query):
    filename = "buscas.csv"
    if os.path.exists(filename):
        df = pd.read_csv('buscas.csv')
        dic = pre_processar(query)
        for word in dic.keys():
            if word in df:
                df[word] = df[word] + 1
            else:
                df[word] = 1
        df.to_csv(filename, index=False)
    else:
        dic = pre_processar(query)
        df = pd.DataFrame(dic, index=['0',])
        df.to_csv(filename, index=False)

In [161]:
def top_10_buscas():
    filename = "buscas.csv"
    if os.path.exists(filename):
        df = pd.read_csv('buscas.csv')
        dic = df.to_dict(orient='split')
        dic = dict(zip(dic['columns'], dic['data'][0]))
        return sorted(dic.items(), key=lambda kv: kv[1], reverse=True)
    return []

In [141]:
df = pd.read_csv('buscas.csv')
df

,ola,amigo,camara
0,2,1,1


In [162]:
dic = top_10_buscas()
dic

[('camara', 3), ('ola', 2), ('amigo', 1)]

In [147]:
contar_busca("camara")

In [2]:
URL = "https://g1.globo.com/index/feed/pagina-{0}.ghtml"

In [3]:
def save(filename, df):
    if os.path.exists(filename):
        df.to_csv(filename, mode='a', index=False, header=False)
    else:
        df.to_csv(filename, index=False)

In [4]:
def gravar(dados):
    df = pd.DataFrame(dados, columns=['link', 'img_url', 'titulo', 'subtitulo', 'descricao'])
    save('noticias_globo.csv', df)

In [5]:
#904
def buscar_noticias():
    dados = []

    for i in range(999, 2000):
        print(f"Página: {i}", end="\r")
        html = requests.get(URL.format(i)).text
        soup = BeautifulSoup(html, 'html.parser')
        for d in soup.find_all('div', class_='bastian-feed-item'):
            title = d.find('a').string
            link = ""
            try:
                link = d.find('a')['href']
            except:
                continue
            subtitle = ""
            try:
                subtitle = d.find_all('div', class_='feed-post-body-resumo')[0].string
            except:
                subtitle = ""
            html = ""
            try:
                html = requests.get(link).text
            except:
                continue
            soup = BeautifulSoup(html, 'html.parser')
            img = ""
            try:
                d = soup.find_all('img', class_="image")[0]
                img = d['data-src']
            except:
                img = "https://s.glbimg.com/jo/g1/static/live/imagens/img_facebook.png"
            content = ""
            
            try:
                docs = soup.find_all('div', class_="wall protected-content")[0]
                for i in docs.find_all('div', class_="mc-column"):
                    if i.find('p') != None and i.find('p').string != None:
                        content += str(i.find('p').string)
            except:
                continue
            dados.append((link, img, title, subtitle, content))
            if len(dados) > 250:
                gravar(dados)
                dados = []
    return "Ok"

In [6]:
buscar_noticias()

'Ok'